## 第9章 模型设定与数据问题

### 9.4 解释变量个数的选择

In [1]:
import pandas as pd
import statsmodels.api as sm

icecream = pd.read_stata('../2_Data/Data-2e/icecream.dta')
icecream['temp_l1'] = icecream['temp'].shift(1)# 引入temp滞后1期
icecream['temp_l2'] = icecream['temp'].shift(2)# 引入temp滞后2期

def Ols_icecream(data, X_cols:list, y_col:str,lags=0):
    X =  icecream[X_cols]
    y =  icecream[y_col]
    Model = sm.OLS(y[lags:], X[lags:])
    Results = Model.fit()
    return Results

#### 1.IC方法

回归后使用信息准则AIC和BIC，观察AIC和BIC的变化，取最小值对应的模型。

结果：
- 引入滞后1期时，IC最小，因此，解释变量中应引入滞后1期。

In [2]:
Results = Ols_icecream(icecream,
                       ['temp', 'price', 'income'], 
                       'consumption')
Results1 = Ols_icecream(icecream,
                        ['temp', 'temp_l1', 'price', 'income'], 
                        'consumption',
                        1)
Results2 = Ols_icecream(icecream,
                        ['temp', 'temp_l1', 'temp_l2', 'price', 'income'], 
                        'consumption',
                        2)

# 直接调用OLSResults的属性
print(f'【不引入temp滞后项】AIC:{Results.aic:.4f}  BIC:{Results.bic:.4f}') 
print(f'【引入temp滞后1期】AIC:{Results1.aic:.4f}  BIC:{Results1.bic:.4f}') 
print(f'【引入temp滞后2期】AIC:{Results2.aic:.4f}  BIC:{Results2.bic:.4f}') 

【不引入temp滞后项】AIC:-110.6299  BIC:-106.4263
【引入temp滞后1期】AIC:-118.0386  BIC:-112.5694
【引入temp滞后2期】AIC:-111.4727  BIC:-104.8117


#### 2. 序贯t规则

步骤：
- 设置一个最大滞后期，引入所有滞后期进行回归
- 对最大滞后期进行t检验，如不显著就向前推
- 直到找到现在的滞后期为止

结果：
1. temp_l2的p值为0.5374,不显著,不接受此最大滞后项,继续检验p-1期。
2. temp_l1的p值为0.0071,显著,接此最大滞后项。


In [3]:
Results2 = Ols_icecream(icecream,
                        ['temp', 'temp_l1', 'temp_l2', 'price', 'income'], 
                        'consumption',
                        2)

if Results2.pvalues['temp_l2']>0.05:
    print(f"temp_l2的p值为{Results2.pvalues['temp_l2']:.4f},不显著,不接受此最大滞后项,继续检验p-1期。")
else:
    print(f"temp_l2的p值为{Results2.pvalues['temp_l2']:.4f},显著,接此最大滞后项。")

temp_l2的p值为0.5374,不显著,不接受此最大滞后项,继续检验p-1期。


In [4]:
Results1 = Ols_icecream(icecream,
                        ['temp', 'temp_l1', 'price', 'income'], 
                        'consumption',
                        1)

if Results1.pvalues['temp_l1']>0.05:
    print(f"temp_l1的p值为{Results1.pvalues['temp_l1']:.4f},不显著,不接受此最大滞后项,继续检验p-1期。")
else:
    print(f"temp_l1的p值为{Results1.pvalues['temp_l1']:.4f},显著,接此最大滞后项。")

temp_l1的p值为0.0071,显著,接此最大滞后项。


### 9.5 对函数形式的检验

In [5]:
import pandas as pd
import statsmodels.api as sm

grilic = pd.read_stata('../2_Data/Data-2e/grilic.dta')

In [6]:
X = grilic[['s','expr','tenure','smsa','rns']]
y = grilic['lnw']
model = sm.OLS(y,sm.add_constant(X))
Results = model.fit()
print(Results.params)

const     4.103675
s         0.102643
expr      0.038119
tenure    0.035615
smsa      0.139667
rns      -0.084080
dtype: float64


使用拟合值的高次项进行RESET检验，linear_reset()的参数中，test_type参数设置为：
- fitted 拟合值,默认值
- exog  所有解释变量
- princomp 解释变量中的第一个主成分

1.先检测是否遗漏拟合值的高次遗漏项

In [7]:
reset_res = sm.stats.diagnostic.linear_reset(Results,
                                             power=[2,3,4], # 测试2, 3, 4次项加入解释变量 
                                             use_f=True # 使用F统计量
                                             )
reset_res.summary()

'<F test: F=1.5068418352326869, p=0.21140502019856158, df_denom=749, df_num=3>'

接受原假设，即高次项系数为0，就是没有遗漏高次项。

2.再检测是否有解释变量的高次遗漏项

In [8]:
reset_res = sm.stats.diagnostic.linear_reset(Results,
                                             test_type='exog',
                                             power=[2,3,4], # 测试2, 3, 4次项加入解释变量 
                                             use_f=True # 使用F统计量
                                             )
reset_res.summary()

'<F test: F=2.03045143326775, p=0.03360894633210033, df_denom=743, df_num=9>'

拒绝了原假设, 说明有遗漏高次项情况，但是并不知道是哪个解释变量导致了这一结果。继续对解释变量的首要影响因素进行检测。

不知道哪个是主要成分，如何识别？
- 对所有解释变量都进行循环检验
  - 如果高次项系数显著（t检验p值 < 0.05）
  - 所有解释变量的RESET检验结果不显著(p值 > 0.05)

则可认为应该加入这个解释变量的高次项。

测试的结果是，这个思路不行。。。

In [9]:
reset_res = sm.stats.diagnostic.linear_reset(Results,
                                             test_type='princomp',
                                             power=[2,3,4], # 测试2, 3, 4次项加入解释变量 
                                             use_f=True # 使用F统计量
                                             )
reset_res.summary()

'<F test: F=4.4748085700701195, p=0.004003746624692338, df_denom=749, df_num=3>'

In [10]:

icecream = pd.read_stata('../2_Data/Data-2e/grilic.dta')

def reset(dataset, X_cols, y_col):
    """
    对拟合值、全体解释变量、不同的解释变量进行RESET检测.
    并显示可识别检测对象的判断结果
    """
    X = dataset[X_cols]
    y = dataset[y_col]
    X = sm.add_constant(X)
    Results_y = sm.OLS(y, X).fit()
    reset_y = sm.stats.diagnostic.linear_reset(Results,
                                                 power=[2,3,4],use_f=True)
    reset_X = sm.stats.diagnostic.linear_reset(Results, test_type='exog',
                                                 power=[2,3,4],use_f=True)
    reset_prin = sm.stats.diagnostic.linear_reset(Results,
                                                         test_type='princomp',
                                                         power=[2,3,4],use_f=True)
    print("被解释变量的RESET检测结果:",reset_y)
    print("全体解释变量的RESET检测结果:",reset_X)
    print('解释变量主成分的RESET检测结果:',reset_prin)
    x2_list = []
    
    for i in X_cols:
        i2 = i+'2'
        dataset[i2] = dataset[i]**2
        X = dataset[X_cols+[i2]]
        # print(X_cols+[i2])
        Results_new = sm.OLS(y, sm.add_constant(X)).fit()
        print(Results_new.params[i2])
        reset_new = sm.stats.diagnostic.linear_reset(Results_new,
                                                         test_type='princomp',
                                                         power=[2,3,4],use_f=True)
        print(f'加入{i}的2次项的RESET检测结果:',reset_new.pvalue)
        if reset_new.pvalue < 0.05 and Results_new.params[i2] > 0.05:
            x2_list.append(i2)
    
    print(f'加入2次项后,可考虑引入{x2_list}:')    
        
reset(dataset=grilic, X_cols=['s','expr','tenure','smsa','rns'], y_col='lnw')    

被解释变量的RESET检测结果: <F test: F=1.5068418352326869, p=0.21140502019856158, df_denom=749, df_num=3>
全体解释变量的RESET检测结果: <F test: F=2.03045143326775, p=0.03360894633210033, df_denom=743, df_num=9>
解释变量主成分的RESET检测结果: <F test: F=4.4748085700701195, p=0.004003746624692338, df_denom=749, df_num=3>
0.00033273358687158224
加入s的2次项的RESET检测结果: 0.03038086400306814
0.0051818814916269615
加入expr的2次项的RESET检测结果: 0.004037117770440055
0.007679465049032812
加入tenure的2次项的RESET检测结果: 0.04327193516213031
0.06983331759564565
加入smsa的2次项的RESET检测结果: 0.2823485794581052
-0.04203987029284524
加入rns的2次项的RESET检测结果: 0.04747189915845505
加入2次项后,可考虑引入[]:


### 9.6 多重共线性

可能涉及到Stata与Python内部的计算机制不一样，同样的数据计算的结果不一样，但是不影响判断。
在后面的计算中，pd.read_stata()函数读取Stata文件时，会出现值较小的数据使用int8类型的情况，这样会造成在计算过程中数据不足的情况，这样会导致计算结果出现大幅偏差的情况。

解决方法：
- 读取数据后直接全部提成64位浮点数，或者使用pandas的astype()函数转换数据类型。

In [38]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

def vif(exog, criterion=5):
    '''vif df格式输出
    计算解释变量的方差膨胀因子和容忍度,其中解释变量不含有常数项
    返回：
        解释变量vif值、容忍度的dataframe 
    Arguments:
        exog -- 解释变量:dataframe
        criterion -- 方差膨胀因子阈值,默认5
    '''
    multicol = pd.DataFrame()
    multicol['变量'] = exog.columns
    multicol['方差膨胀因子'] = [variance_inflation_factor(exog.values, i) for i in range(exog.shape[1])]
    multicol['容忍度'] = 1/multicol['方差膨胀因子']
    _ = []
    for i in multicol['方差膨胀因子']:
        if i > criterion:
            _.append('**是**')
        else:
            _.append('否')    
    multicol['是否多重共线'] = _
    return multicol

grilic = pd.read_stata('../2_Data/Data-2e/grilic.dta')
X = grilic[['s','expr','tenure','smsa','rns']]
vif_1 = vif(X)
# for i in vif:
#     print(vif[i].dtype)
vif_1

,变量,方差膨胀因子,容忍度,是否多重共线
0,s,4.973077,0.201083,否
1,expr,1.698322,0.588816,否
2,tenure,2.288772,0.436916,否
3,smsa,3.417113,0.292645,否
4,rns,1.364364,0.732942,否


引入平方项，易引起多重共线。

注意：这里出现了一个数据类型转换的问题

例如：

In [41]:
# 转换数据类型
cols_to_convert = X.select_dtypes(include=['int8'])
for col in cols_to_convert:
    X[col] = X[col].astype('float32')

# 生成s的平方项
X['s2'] = X['s'] ** 2
X

C:\Users\NINGMEI\AppData\Local\Temp\ipykernel_27916\1584946888.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('float32')
C:\Users\NINGMEI\AppData\Local\Temp\ipykernel_27916\1584946888.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('float32')
C:\Users\NINGMEI\AppData\Local\Temp\ipykernel_27916\1584946888.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

,s,expr,tenure,smsa,rns,s2
0,12.0,0.462,0.0,1.0,0.0,144.0
1,16.0,0.000,2.0,1.0,0.0,256.0
2,14.0,0.423,1.0,1.0,0.0,196.0
3,12.0,0.333,1.0,1.0,0.0,144.0
4,9.0,9.013,3.0,1.0,0.0,81.0
...,...,...,...,...,...,...
753,16.0,0.000,1.0,1.0,1.0,256.0
754,12.0,0.692,1.0,0.0,1.0,144.0
755,12.0,4.828,0.0,0.0,1.0,144.0
756,12.0,2.489,2.0,0.0,1.0,144.0


In [42]:
vif_2 = vif(X)
vif_2

,变量,方差膨胀因子,容忍度,是否多重共线
0,s,60.913912,0.016417,**是**
1,expr,1.857611,0.538326,否
2,tenure,2.328171,0.429522,否
3,smsa,3.502414,0.285517,否
4,rns,1.401529,0.713506,否
5,s2,46.852452,0.021344,**是**


结果显示s和s2都有多重共线性

In [46]:
X['sd'] = (X['s']-X['s'].mean())/X['s'].std()
X['sd2'] = X['sd']**2
for i in ['s','s2']:
    if i in X:   
        X.drop(i, axis=1, inplace=True)
X

C:\Users\NINGMEI\AppData\Local\Temp\ipykernel_27916\3802650683.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['sd'] = (X['s']-X['s'].mean())/X['s'].std()
C:\Users\NINGMEI\AppData\Local\Temp\ipykernel_27916\3802650683.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['sd2'] = X['sd']**2
C:\Users\NINGMEI\AppData\Local\Temp\ipykernel_27916\3802650683.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

,expr,tenure,smsa,rns,sd,sd2
0,0.462,0.0,1.0,0.0,-0.629535,0.396314
1,0.000,2.0,1.0,0.0,1.162718,1.351913
2,0.423,1.0,1.0,0.0,0.266592,0.071071
3,0.333,1.0,1.0,0.0,-0.629535,0.396314
4,9.013,3.0,1.0,0.0,-1.973724,3.895586
...,...,...,...,...,...,...
753,0.000,1.0,1.0,1.0,1.162718,1.351913
754,0.692,1.0,0.0,1.0,-0.629535,0.396314
755,4.828,0.0,0.0,1.0,-0.629535,0.396314
756,2.489,2.0,0.0,1.0,-0.629535,0.396314


In [47]:
vif(X)

,变量,方差膨胀因子,容忍度,是否多重共线
0,expr,1.804305,0.554230,否
1,tenure,2.038801,0.490484,否
2,smsa,2.079105,0.480976,否
3,rns,1.271753,0.786316,否
4,sd,1.299687,0.769416,否
5,sd2,2.029292,0.492783,否
